In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
stop_words = set(stopwords.words('english')) 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras.layers import concatenate,CuDNNLSTM
from keras import initializers
from keras import backend as K
import random

from keras.layers import Dense, LSTM,Dropout, Activation,Bidirectional,Reshape,Permute,Multiply,Flatten,Lambda,Layer,TimeDistributed,CuDNNLSTM

In [0]:
def SentenceToPOS(sentences):
    data = []
    for sentence in sentences: 
        one_sentence_pos = [] 
        
        wordsList = nltk.word_tokenize(sentence)
        
        wordsList = [w for w in wordsList if not w in stop_words]

        tagged = nltk.pos_tag(wordsList)
        
        for val in tagged:
            one_sentence_pos.append(val[1])
        data.append(one_sentence_pos)
    return data

def Fianl_X_and_Y(novelPOS, novelLabel, sentenceSize):
    i, X, Y = 0, [], []
    for novel in novelPOS:
        start, end = 0, sentenceSize
        while(end <= len(novel)):
            X.append(novel[start:end])
            Y.append(novelLabel[i])
            start = end
            end += sentenceSize 
        i += 1
    return X, Y

def Get_X_and_Y_In_POS_Form(fullPath, sentenceSize):
    filenameList = os.listdir(fullPath)
    filenameList.sort()
    
    novelPOS, novelLabel, fileNumber = [], [], 1
    for filename in filenameList:
        print(fileNumber,"/",len(filenameList)," Done")
        fileNumber += 1
        Path = fullPath + filename
        with open(Path, 'r') as f:
            data = f.read().replace('"\n"','').replace('\n',' ').replace('- ','')
            sentences = sent_tokenize(data)
            POS = SentenceToPOS(sentences)
            novelPOS.append(POS)
            novelLabel.append(filename[8])
    return Fianl_X_and_Y(novelPOS, novelLabel, sentenceSize)

In [0]:
print("Preprocessing Phase 1 : Fetching data")
print(".....................................")

X, Y = Get_X_and_Y_In_POS_Form('/content/gdrive/My Drive/dataset/',100)

print("Preprocessing Phase 1 : Finished\n\n")

In [0]:
maxval, minval = 0, 999999
for doc in X:
  for sentence in doc:
    minval = min(minval, len(sentence))
    maxval = max(maxval, len(sentence))
print(maxval, minval)

In [0]:
Labels = np.unique(Y, return_counts = True)[0]

In [0]:
LabelCount = {}
for label in Labels:
  LabelCount[label] = 0

In [0]:
MIN_VAL = min(np.unique(Y,return_counts=True)[1]) + 200

In [0]:
def dataBalancing(X, Y, LabelCount, MIN_VAL):
  X_data = []
  for i in range(len(X)):
      if(LabelCount[Y[i]] < MIN_VAL):
          X_data.append([X[i],Y[i]])
          LabelCount[Y[i]] += 1

  random.shuffle(X_data)

  new_X = []
  new_Y = []
  for m ,n in X_data:
    new_X.append(m)
    new_Y.append(n)

  return new_X, new_Y

In [0]:
X, Y = dataBalancing(X, Y, LabelCount, MIN_VAL)

In [0]:
def tag_to_index_dictionary(X):
    tag = set([])
    for doc in X:
        for sentence in doc:
            for word in sentence:
                tag.add(word)
    tag2index = {t: i + 1 for i, t in enumerate(list(tag))}
    tag2index['-PAD-'] = 0
    return tag2index

def convert_tag_to_sequence_numbers(X, tag2index):
    new_X=[]
    for doc in X:
        new_S = []
        for sentence in doc:
            new_W = []
            for word in sentence:
                new_W.append(tag2index[word])
            new_S.append(new_W)
        new_X.append(new_S)
    return new_X


def pad_zeros_to_sequence(X, max_length):
    new_X = []
    for doc in X:
        new_X.append(pad_sequences(doc, maxlen=max_length, padding='post'))
    return new_X

def Encode_Labels(Y):
    le = LabelEncoder()
    return le.fit_transform(Y)

In [0]:
tag2index = tag_to_index_dictionary(X)
DICTIONARY_LENGTH = len(tag2index)
MAX_LENGTH = 30 # why 30 only ?
SENTENCES_IN_NOVEL = 100

X = convert_tag_to_sequence_numbers(X, tag2index)
X = pad_zeros_to_sequence(X, MAX_LENGTH)
X = np.array(X)
TOTAL_ROWS = X.shape[0]